In [1]:
!pip install backtrader
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 4.7 MB/s eta 0:00:00


In [2]:
import backtrader as bt
import math
import yfinance as yf

추세전략

In [9]:
class BollingerBands(bt.Strategy):
  params = dict(
      bb_period=20, # 볼린저 밴드 기간
      bb_devfactor=1, # 볼린저 밴드 표준편차
      ma_period=50 # 이동평균선 기간
  )

  def __init__(self):
    # 볼린저밴드
    self.bb = bt.indicators.BollingerBands(self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor)
    # 이동평균선
    self.ma = bt.indicators.MovingAverageSimple(self.data.close, period=self.params.ma_period)
    # 매수 신호 추적 변수
    self.order = None

  def next(self):
    if self.order:  # 만약 주문이 있으면 대기
      return
    # 포지션이 없을 때(시장 미진입 상태)
    if not self.position:
      # 매수 : 종가가 볼린저 밴드 상단선을 돌파
      if self.data.close[0] > self.bb.lines.top[0] and self.data.close[-1] <= self.bb.lines.top[-1]:
        order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
        self.buy(size=order_size)

    # 포지션이 있을 때(시장 진입 상태)
    else:
      # 종가가 이평선 하향 돌파 -> 청산
      if self.data.close[0] < self.ma[0] and self.data.close[-1] >= self.ma[-1]:
        self.close()


  def log(self, message):
    print(message)

  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      # Buy/Sell order submitted/accepted to/by broker - Nothing to do
      return

    # Check if an order has been completed
    # Attention: broker could reject order if not enough cash
    cur_date = None
    if order.status in [order.Completed]:
      cur_date = order.data.datetime.date(0)
      if order.isbuy():
        self.log(
            f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      elif order.issell():
        self.log(
            f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log(
          f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')

In [10]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBands)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-04-02 [매수 주문 실행] 종목:  	 수량: 1498 	 가격: 6606.78
2020-06-20 [매도 주문 실행] 종목:  	 수량: -1498 	 가격: 9290.96
2020-07-09 [매수 주문 실행] 종목:  	 수량: 1464 	 가격: 9427.99
2020-07-10 [매도 주문 실행] 종목:  	 수량: -1464 	 가격: 9273.36
2020-07-22 [매수 주문 실행] 종목:  	 수량: 1440 	 가격: 9375.08
2020-09-04 [매도 주문 실행] 종목:  	 수량: -1440 	 가격: 10230.37
2020-10-09 [매수 주문 실행] 종목:  	 수량: 1341 	 가격: 10927.91
2021-04-19 [매도 주문 실행] 종목:  	 수량: -1341 	 가격: 56191.59
2021-05-09 [매수 주문 실행] 종목:  	 수량: 1266 	 가격: 58877.39
2021-05-11 [매도 주문 실행] 종목:  	 수량: -1266 	 가격: 55847.24
2021-06-14 [매수 주문 실행] 종목:  	 수량: 1796 	 가격: 39016.97
2021-09-11 [매도 주문 실행] 종목:  	 수량: -1796 	 가격: 44869.84
2021-10-02 [매수 주문 실행] 종목:  	 수량: 1666 	 가격: 48137.47
2021-11-19 [매도 주문 실행] 종목:  	 수량: -1666 	 가격: 56896.13
2021-12-24 [매수 주문 실행] 종목:  	 수량: 1853 	 가격: 50806.05
2022-02-12 [매도 주문 실행] 종목:  	 수량: -1853 	 가격: 42412.30
2022-02-16 [매수 주문 실행] 종목:  	 수량: 1754 	 가격: 44578.28
2022-02-18 [매도 주문 실행] 종목:  	 수량: -1754 	 가격: 40552.13
2022-03-02 [매수 주문 실행] 종목:  	 수량: 1595 	 가격

스탑로스 10% 설정

In [11]:
class BollingerBandsStopLoss(bt.Strategy):
  params = dict(
      bb_period=20, # 볼린저 밴드 기간
      bb_devfactor=1, # 볼린저 밴드 표준편차
      ma_period=50, # 이동평균선 기간
      stop_loss=0.1, # 손절 비율(10%)
  )

  def __init__(self):
    # 볼린저밴드
    self.bb = bt.indicators.BollingerBands(self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor)
    # 이동평균선
    self.ma = bt.indicators.MovingAverageSimple(self.data.close, period=self.params.ma_period)
    # 매수 신호 추적 변수
    self.order = None
    # 매수 가격 저장
    self.buy_price = None

  def next(self):
    if self.order:  # 만약 주문이 있으면 대기
      return
    # 포지션이 없을 때(시장 미진입 상태)
    if not self.position:
      # 매수 : 종가가 볼린저 밴드 상단선을 돌파
      if self.data.close[0] > self.bb.lines.top[0] and self.data.close[-1] <= self.bb.lines.top[-1]:
        order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
        self.buy(size=order_size)
        self.buy_price = self.data.close[0]

    # 포지션이 있을 때(시장 진입 상태)
    else:
      # 종가가 이평선 하향 돌파 or 매수가 대비 10% 하락 -> 청산
      stop_loss_price = self.buy_price * (1 - self.params.stop_loss) # 손절가 계산
      if (self.data.close[0] < stop_loss_price) or (self.data.close[0] < self.ma[0] and self.data.close[-1] >= self.ma[-1]):
        self.close()
        self.buy_price = None # 매도 후 매수 가격 초기화


  def log(self, message):
    print(message)

  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      # Buy/Sell order submitted/accepted to/by broker - Nothing to do
      return

    # Check if an order has been completed
    # Attention: broker could reject order if not enough cash
    cur_date = None
    if order.status in [order.Completed]:
      cur_date = order.data.datetime.date(0)
      if order.isbuy():
        self.log(
            f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      elif order.issell():
        self.log(
            f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log(
          f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


In [12]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBandsStopLoss)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-04-02 [매수 주문 실행] 종목:  	 수량: 1498 	 가격: 6606.78
2020-06-20 [매도 주문 실행] 종목:  	 수량: -1498 	 가격: 9290.96
2020-07-09 [매수 주문 실행] 종목:  	 수량: 1464 	 가격: 9427.99
2020-07-10 [매도 주문 실행] 종목:  	 수량: -1464 	 가격: 9273.36
2020-07-22 [매수 주문 실행] 종목:  	 수량: 1440 	 가격: 9375.08
2020-09-04 [매도 주문 실행] 종목:  	 수량: -1440 	 가격: 10230.37
2020-10-09 [매수 주문 실행] 종목:  	 수량: 1341 	 가격: 10927.91
2021-04-19 [매도 주문 실행] 종목:  	 수량: -1341 	 가격: 56191.59
2021-05-09 [매수 주문 실행] 종목:  	 수량: 1266 	 가격: 58877.39
2021-05-11 [매도 주문 실행] 종목:  	 수량: -1266 	 가격: 55847.24
2021-06-14 [매수 주문 실행] 종목:  	 수량: 1796 	 가격: 39016.97
2021-06-22 [매도 주문 실행] 종목:  	 수량: -1796 	 가격: 31622.38
2021-07-25 [매수 주문 실행] 종목:  	 수량: 1653 	 가격: 34290.29
2021-09-11 [매도 주문 실행] 종목:  	 수량: -1653 	 가격: 44869.84
2021-10-02 [매수 주문 실행] 종목:  	 수량: 1530 	 가격: 48137.47
2021-11-19 [매도 주문 실행] 종목:  	 수량: -1530 	 가격: 56896.13
2021-12-24 [매수 주문 실행] 종목:  	 수량: 1701 	 가격: 50806.05
2022-01-06 [매도 주문 실행] 종목:  	 수량: -1701 	 가격: 43565.51
2022-02-05 [매수 주문 실행] 종목:  	 수량: 1777 	 가격

스탑로스 10%로 설정 + 윌리엄스의 리스크 고정

In [13]:
import math
import backtrader as bt

class BollingerBandsStopLossRiskFixed(bt.Strategy):
    params = dict(
        bb_period=20,         # 볼린저 밴드 기간
        bb_devfactor=1,       # 볼린저 밴드 표준편차
        ma_period=50,         # 이동평균선 기간
        stop_loss=0.1,        # 손절 비율 (10%)
        risk_percent=0.03,    # 리스크 비율 (3%)
    )

    def __init__(self):
        # 볼린저 밴드 지표
        self.bb = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.bb_period,
            devfactor=self.params.bb_devfactor
        )
        # 이동평균선 지표
        self.ma = bt.indicators.MovingAverageSimple(
            self.data.close,
            period=self.params.ma_period
        )
        self.order = None
        self.buy_price = None

    def next(self):
        if self.order:  # 아직 미체결 주문이 있으면 대기
            return

        # 포지션이 없을 때: 진입 조건
        if not self.position:
            # 종가가 볼린저 밴드 상단선을 돌파할 때 매수
            if self.data.close[0] > self.bb.lines.top[0] and self.data.close[-1] <= self.bb.lines.top[-1]:
                self.buy_price = self.data.close[0]
                # 계좌 가치의 risk_percent 만큼의 리스크 금액 산정
                risk_amount = self.broker.getvalue() * self.params.risk_percent
                # 손절가: 매수 가격의 (1 - stop_loss) 비율
                stop_loss_price = self.buy_price * (1 - self.params.stop_loss)
                # 포지션 사이즈 계산 (리스크 금액 / (매수가 - 손절가))
                position_size = risk_amount / (self.buy_price - stop_loss_price)
                position_size = (position_size)
                if position_size <= 0:
                    return

                # 브라켓 주문 사용: 매수 주문과 동시에 스탑로스 주문을 부모 주문에 연계
                parent_order = self.buy(size=position_size)
                self.sell(exectype=bt.Order.Stop,
                          parent=parent_order,
                          price=stop_loss_price,
                          size=position_size)
        # 포지션 보유 중일 때: 청산 조건
        else:
            # 이동평균선 하향 돌파 시 전체 포지션 청산
            if self.data.close[0] < self.ma[0] and self.data.close[-1] >= self.ma[-1]:
                self.close()
                self.buy_price = None

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 주문 제출/수락 상태이면 별도 처리 없음
            return

        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
            )


In [14]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBandsStopLossRiskFixed)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-04-02 [매수 주문 실행] 종목:  	 수량: 454.0792412619682 	 가격: 6606.78
None 주문이 거부되었습니다. 종목:  	 수량: 454.0792412619682 	 가격: 6606.78
2020-06-20 [매도 주문 실행] 종목:  	 수량: -454.0792412619682 	 가격: 9290.96
2020-07-09 [매수 주문 실행] 종목:  	 수량: 356.28277695210795 	 가격: 9427.99
None 주문이 거부되었습니다. 종목:  	 수량: 356.28277695210795 	 가격: 9427.99
2020-07-10 [매도 주문 실행] 종목:  	 수량: -356.28277695210795 	 가격: 9273.36
2020-07-22 [매수 주문 실행] 종목:  	 수량: 355.9112153767755 	 가격: 9375.08
None 주문이 거부되었습니다. 종목:  	 수량: 355.9112153767755 	 가격: 9375.08
2020-09-04 [매도 주문 실행] 종목:  	 수량: -355.9112153767755 	 가격: 10230.37
2020-10-09 [매수 주문 실행] 종목:  	 수량: 313.46353050571014 	 가격: 10927.91
None 주문이 거부되었습니다. 종목:  	 수량: 313.46353050571014 	 가격: 10927.91
2021-04-19 [매도 주문 실행] 종목:  	 수량: -313.46353050571014 	 가격: 56191.59
2021-05-09 [매수 주문 실행] 종목:  	 수량: 130.25161512558586 	 가격: 58877.39
None 주문이 거부되었습니다. 종목:  	 수량: 130.25161512558586 	 가격: 58877.39
2021-05-11 [매도 주문 실행] 종목:  	 수량: -130.25161512558586 	 가격: 55847.24
2021-06-14 [매수 주문 실행] 종목